In [ ]:
import pandas as pd
from collections import  Counter
import re


import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from gensim.parsing.preprocessing import remove_stopwords

import spacy
sp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/Shareddrives/Text Analytics/HW2/')
os.getcwd()

'/content/drive/Shareddrives/Text Analytics/HW2'

In [ ]:
from lxml import etree, objectify

# Read the dictionary file.
Parser = objectify.makeparser(recover=True)
Tree = objectify.fromstring(''.join(open('dictionary.xml').readlines()), Parser)


# function: 
def getSenses(word, pos):
    global Tree
    item = Tree.xpath("//lexelt[@item='%s.%s']" % (word, pos))    
    senses = []
    for sense in item[0].getchildren():
        senses.append(dict(zip(sense.keys(), sense.values())))
    return senses


# function:
def getSense(word, pos, sense_id):
    global Tree
    sense = Tree.xpath("//lexelt[@item='%s.%s']/sense[@id='%d']" % (word, pos, sense_id))
    return dict(zip(sense[0].keys(), sense[0].values()))


# Example
print(getSense('begin', 'v', 2))
print(getSenses('begin', 'v'))

{'id': '2', 'wordnet': '1,8,5', 'gloss': 'initiate an undertaking', 'examples': "They'll begin their tour with a concert in London. | You should begin the stew now so it will have four hours to simmer. | The government is beginning to protect the coral reefs of that island. | He was just beginning a novel when the phone rang. | Acme Tire Co. began offering their employees health insurance. | John has begun to take them seriously. | She began ballet at age four."}
[{'id': '1', 'wordnet': '2,3,5,6,7', 'gloss': 'start, have an initial point ', 'examples': "Prices for these homes begin at $250,000 | The war began on a Thursday. | She began to feel sick. | When life began was there oxygen in the atmosphere? | The novel begins with a shipwreck at sea. | It's beginning to look like rain. | His property begins at the fence."}, {'id': '2', 'wordnet': '1,8,5', 'gloss': 'initiate an undertaking', 'examples': "They'll begin their tour with a concert in London. | You should begin the stew now so it

In [ ]:
temp=getSenses('begin', 'v')

In [ ]:
temp[0].keys()

dict_keys(['id', 'wordnet', 'gloss', 'examples'])

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(getSenses('work', 'v'))

[{'id': '1', 'wordnet': '1,2,6,8,10,12', 'gloss': 'exert oneself in an activity', 'examples': "He is working hard to bring up his grade. | They worked on a farm growing up. | Those men will work for minimum wage. | They work us hard in that gym class. | The salesman works the Midwest. | Don't work those children so hard. | Her fingers worked with incredible speed. | She worked at the algebra problems until she'd solved them all. | At the end of sentence, he had worked off his debt to society. | Please be quiet while I work on the taxes."}, {'id': '2', 'wordnet': '3,4,11', 'gloss': 'perform, function, behave', 'examples': "How will the parachute actually work when deployed? | This old radio doesn't work anymore. | That drug has worked miracles for those with arthritis."}, {'id': '3', 'wordnet': '5,13,14,20,22', 'gloss': 'shape, form, handle', 'examples': "The artist taught them to work different kinds of clay. | We'll have to work this hard soil a lot before planting lettuce. | The doug

In [ ]:
temp=getSenses('come', 'v')
import string

In [ ]:
nltk.download('wordnet')
from nltk import wordnet as wn
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from nltk.util import ngrams
import string
import math

In [ ]:
def data_cleaning(tokenized_data):
  ds = tokenized_data
  for idx,j in enumerate(ds):
    if j not in stopwords.words():
      j = j.translate(str.maketrans('', '', string.punctuation+'0123456789'))  #### removing punctuations
      j = j.lower() #### lowering all words ######
      j = lemmatizer.lemmatize(j) #### lemmatize noun
      j = lemmatizer.lemmatize(j,'v') #### lemmatize verb
      ds[idx] = j
  ds = [j for j in ds if not j in stopwords.words()]
  ds = list(filter(None, ds)) ### filter out empty tokens if any
  return ds

In [ ]:
### checking
data_cleaning(['control,', 'exploited,', 'act', 'upon', 'She', 'worked', 'on', 'her', 'friends', 'to', 'support'])

['control', 'exploit', 'act', 'upon', 'work', 'friend', 'support']

# Original lesk (modified) - high rewards for consecutive overlaps

In [ ]:
def original_lesk(sentence,word,pos,modified=False):
  max_common = 0
  id=0
  sense = None
  try:
    temp=getSenses(word,pos)  #### sense from dictionary
    context = sentence#.split() ##### any random sentence or data
    #print(context)
    context = context.lower().translate(str.maketrans('', '', string.punctuation+'0123456789'))
    context_tags = nltk.pos_tag(context.split())   ### pos tags of those words which are not provided before data cleaning
    con = []
    for i in range(len(context_tags)):
      l = context_tags[i]
      k = [j.split() for j in l[1]][0]
      x = l[0].lower().translate(str.maketrans('', '', string.punctuation+'0123456789'))
      x = lemmatizer.lemmatize(x)
      x = lemmatizer.lemmatize(x,'v')
      d = dict(word=x,pos=k[0].lower())
      con.append(d)
    #print(con)
    for j in range(len(temp)):
      score=0
      lesk_dict = []
      ### word-meaning of the word###
      lesk_dict+= temp[j]["gloss"].split()
      temp_sense = temp[j]["gloss"]   ### temporary sense ####
      #### using examples as well
      x = [l.split() for l in temp[j]['examples'].split('|')]
      examples = [item for sublist in x for item in sublist] ### examples split
      ##### using gloss + examples
      lesk_dict += examples
      lesk_dict = data_cleaning(lesk_dict)
      #print(lesk_dict)
      for k in range(len(con)):
        if con[k]["word"] != word:
          #print(con[k]["word"])
          try:
            ex = getSenses(con[k]['word'],con[k]['pos'])
            #print(ex)
            new_dict = []
            ### word-meaning of the word###
            for l in range(len(ex)):
              new_dict+= ex[l]["gloss"].split()
            #new_dict+= ex[i]["gloss"].split()
            #print(new_dict)
            #### using examples as well
            t=[]
            for l in range(len(ex)):
              t+= ex[l]["examples"].split('|')
            new_x = [l.split() for l in t]
            new_examples = [item for sublist in new_x for item in sublist] ### examples split
            ##### using gloss + examples
            new_dict += new_examples
            #print(new_dict)
            new_dict = data_cleaning(new_dict)
            #print(new_dict)
            #### score computation from overlapping
            score_set = set(lesk_dict).intersection(new_dict) #### checking common words in gloss and examples
            #print(score_set)
            score += len(score_set)
            if modified:
              score_set_bi = set(list(ngrams(lesk_dict, 2))).intersection(list(ngrams(new_dict, 2))) #### checking common bigrams in gloss and examples
              score+= math.pow(len(score_set_bi),2)   #### rewarding consecutive overlaps by giving score which should be square of such overlaps
            #print(score_set,score)
          except IndexError:
            pass
      temp_id = temp[j]["id"]
      if score > max_common:
        sense = temp_sense   #### updating sentence with most number of overlapping words
        id = temp_id
        max_common = score
  except (IndexError, UnboundLocalError) as e:
    sense,id=(None,0)
  return sense,id

In [ ]:
# random test
sent = 'working by doing things in mexico'
original_lesk(sent,'work','v',modified=True)

('exert oneself in an activity', '1')

Reading Training data

In [ ]:
def file_dataframe(filename):
  input_string = os.getcwd() + '/'+str(filename)
  open_file = open(input_string, 'r')
  read_data = open_file.readlines()
  return read_data 

In [ ]:
### reading training data
train = file_dataframe('train.data')
head = list(train)
head[0:3]

["affect.v | 1 | The Mahatma , or `` great souled one , '' instigated several campaigns of passive resistance against the British government in India . Unfortunately , according to Webster 's Biographical Dictionary , `` His policies went beyond his control and resulted ... in riots and disturbances '' and later a renewed campaign of civil disobedience `` resulted in rioting and a second imprisonment . '' I am not a proponent of everything Gandhi did , but some of his law breaking was justified because India was then under occupation by a foreign power , and Indians were not able to participate fully in decisions that vitally %% affected %% them . It is difficult , however , to justify civil disobedience , non-violent or not , where citizens have full recourse to the ballot box to effect change . Where truly representative governments are safeguarded by constitutional protections of human rights and an independent judiciary to construe those rights , there is no excuse for breaking the

In [ ]:
###### manipulating in same format as in dictionary
def data_handling(filename):
  train = file_dataframe(filename)
  head = list(train)
  text=[]
  for i in range(len(head)):
    word = head[i].split('|')[0].split('.')[0].strip()
    pos= head[i].split('|')[0].split('.')[1].strip()
    id = head[i].split('|')[1].strip()
    example = head[i].split('|')[2]
    text.append(dict(id = id,word=word,pos=pos,examples=example))
  return text

In [ ]:
training_text = data_handling("train.data")

In [ ]:
validation_text = data_handling("validate.data")
len(validation_text)

933

# Checking performance of Original Lesk on validation data

In [ ]:
########### Actual Sense Prediction #######
def actual_sense(data):
  actual_id =[]
  for i in range(len(data)):
    actual_id.append(data[i]['id'])
  return actual_id

In [ ]:
############ Sense Prediction Function ##############
def prediction(data,corpusLesk=False,simpleLesk=False,originalLesk=False,modified=True):
  predicted_id=[]
  for i in range(len(data)):
    if corpusLesk:
      sense,pred_id = corpus_lesk(data[i]["examples"],data[i]['word'],data[i]['pos'],modified)
    elif simpleLesk:
      sense,pred_id = simple_lesk(data[i]["examples"],data[i]['word'],data[i]['pos'],modified)
    elif originalLesk:
      sense,pred_id = original_lesk(data[i]["examples"],data[i]['word'],data[i]['pos'],modified)
    predicted_id.append(pred_id)
    if i%50==0:
      print(i)
  return predicted_id

In [ ]:
###### Warning !!! Might take 45-50 minutes to run ###############
given_sense = actual_sense(validation_text)
pred_sense = prediction(validation_text,originalLesk=True)

In [ ]:
from sklearn.metrics import confusion_matrix
cm_simple=confusion_matrix(given_sense,pred_sense)

In [ ]:
cm_simple.diagonal().sum()/sum(cm_simple).sum()*100    ##### Accuracy ####

41.102301


# Simple Lesk

In [ ]:
def simple_lesk(sentence,word,pos,modified=False):
  max_common = 0
  id=0
  sense = None
  try:
    temp=getSenses(word,pos)  #### sense from dictionary
    context = sentence.split() ##### any random sentence or data
    context = data_cleaning(context) #### cleaning data
    for i in range(len(temp)):
      lesk_dict = []
      ### word-meaning of the word###
      lesk_dict+= temp[i]["gloss"].split()
      temp_sense = temp[i]["gloss"]   ### temporary sense ####
      #### using examples as well
      x = [j.split() for j in temp[i]['examples'].split('|')]
      examples = [item for sublist in x for item in sublist] ### examples split
      ##### using gloss + examples
      lesk_dict += examples
      #print(lesk_dict)
      lesk_dict = data_cleaning(lesk_dict)
      #### score computation from overlapping
      score_set = set(lesk_dict).intersection(context) #### checking common words in gloss and examples
      score = len(score_set)
      if modified:
        score_set_bi = set(list(ngrams(lesk_dict, 2))).intersection(list(ngrams(context, 2))) #### checking common bigrams in gloss and examples
        score+= math.pow(len(score_set_bi),2)   #### rewarding consecutive overlaps by giving score which should be square of such overlaps
      #print(score_set,score_set_bi,score)
      temp_id = temp[i]["id"]
      if score > max_common:
        sense = temp_sense   #### updating sentence with most number of overlapping words
        id = temp_id
        max_common = score
  except IndexError:
    sense,id=(None,0)
  return sense,id

# Checking performance of Simple Lesk (basic+modified) on validation data

Simple Lesk with modified version which takes into account consecutive overlaps

In [ ]:
###### Warning !!! Might take 15-16 minutes to run ###############
given_sense = actual_sense(validation_text)
pred_sense = prediction(validation_text,simpleLesk=True,modified=False) ### Simple lesk basic ###

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900


In [ ]:
from sklearn.metrics import confusion_matrix
cm_simple=confusion_matrix(given_sense,pred_sense)

In [ ]:
cm_simple.diagonal().sum()/sum(cm_simple).sum()*100    ##### Accuracy ####

49.30332261521972

Simple Lesk modified with high reward to consecutive overlaps

In [ ]:
###### Warning !!! Might take 15-16 minutes to run ###############
given_sense = actual_sense(validation_text)
pred_sense = prediction(validation_text,simpleLesk=True,modified=True) ### Simple lesk modified ###

In [ ]:
from sklearn.metrics import confusion_matrix
cm_simple=confusion_matrix(given_sense,pred_sense)
cm_simple

In [ ]:
cm_simple.diagonal().sum()/sum(cm_simple).sum()*100    ##### Accuracy ####

49.51768488745981

# Applying Simple Modified Lesk on Test data

In [ ]:
test_text = data_handling("test.data")
len(test_text)

3918

In [ ]:
given_sense_test = actual_sense(test_text)
pred_test_simple = prediction(test_text,simpleLesk=True)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900


In [ ]:
pred_test_simple[0:10]

['2', '1', '2', '2', '2', '1', '3', '1', '1', '1']

In [ ]:
text_file = open("test_simple_lesk.data", "w")
for i in range(len(test_text)):
  test_text[i]["id"] = pred_test_simple[i]
  line = test_text[i]['word']+'.'+test_text[i]['pos']+" | "+str(test_text[i]['id'])+" | "+test_text[i]['examples']
  text_file.write(line)
text_file.close()

In [ ]:
len(file_dataframe('test_simple_lesk.data'))

3918

In [ ]:
test_text[0]['word']+'.'+test_text[0]['pos']+" | "+test_text[0]['id']+" | "+test_text[0]['examples']

"capital.n | 0 |  Influential members of the House Ways and Means Committee introduced legislation that would restrict how the new savings-and-loan bailout agency can raise capital , creating another potential obstacle to the government 's sale of sick thrifts . The bill , whose backers include Chairman Dan Rostenkowski ( D. , Ill. ) , would prevent the Resolution Trust Corp. from raising temporary working %% capital %% by having an RTC-owned bank or thrift issue debt that would n't be counted on the federal budget . The bill intends to restrict the RTC to Treasury borrowings only , unless the agency receives specific congressional authorization . `` Such agency ` self-help ' borrowing is unauthorized and expensive , far more expensive than direct Treasury borrowing , '' said Rep. Fortney Stark ( D. , Calif. ) , the bill 's chief sponsor .\n"

#Corpus Lesk

In [ ]:
########### Augment train data with dictionary to train ##############
def merge_training(text,word,pos):
  n = list(filter(lambda x: (x['word'] == word and x['pos'] == pos), text))
  dict2 = {dct['id']:dct for dct in n}
  temp=getSenses(word,pos)
  out = []
  for d1 in temp:
    d = dict(**d1)
    if dict2.get(d1['id'])!=None:
      new_ex = d1['examples']+' | '+ dict2.get(d1['id'])['examples']
      d.update(examples=new_ex)
    else:
      d.update(dict2.get(d1['id'],{}))
    out.append(d)
  return out

In [ ]:
def corpus_lesk(sentence,word,pos,modified=False):
  max_common = 0
  id=0
  sense = None
  augmented_data = merge_training(training_text,word,pos)
  try:
    temp=augmented_data
    context = sentence.split() ##### any random sentence or data
    context = data_cleaning(context) #### cleaning data
    for i in range(len(temp)):
      lesk_dict = []
      ### word-meaning of the word###
      lesk_dict+= temp[i]["gloss"].split()
      temp_sense = temp[i]["gloss"]   ### temporary sense ####
      #### using examples as well
      x = [j.split() for j in temp[i]['examples'].split('|')]
      examples = [item for sublist in x for item in sublist] ### examples split
      ##### using gloss + examples
      lesk_dict += examples
      #print(lesk_dict)
      lesk_dict = data_cleaning(lesk_dict)
      #### score computation
      score_set = set(lesk_dict).intersection(context) #### checking common words in gloss and examples
      score = len(score_set)
      if modified:
        score_set_bi = set(list(ngrams(lesk_dict, 2))).intersection(list(ngrams(context, 2))) #### checking common bigrams in gloss ans examples
        score+= math.pow(len(score_set_bi),2)
      #print(score_set,score_set_bi,score)
      temp_id = temp[i]["id"]
      if score > max_common:
        sense = temp_sense   #### updating sentence with most number of overlapping words
        id = temp_id
        max_common = score
  except IndexError:
    sense,id=(None,0)
  return sense,id

In [ ]:
###### random test
sent = 'come to Mexico'
corpus_lesk(sent,'come','v')

('move, travel, arrive', '1')

In [ ]:
#### Checking performance of Corpus lesk

In [ ]:
############## Warning !!!! Will take 30 minutes
pred_sense = prediction(validation_text,corpusLesk=True)

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(given_sense,pred_sense)
#cm

In [ ]:
cm.diagonal().sum()/sum(cm).sum()*100

48.01714898177921

# Running Corpus lesk and saving file

In [ ]:
########## Warning !!!!! Will take 1.30 hours ############
pred_test_corpus = prediction(test_text)

In [ ]:
text_file = open("test_corpus_lesk.data", "w")
for i in range(len(test_text)):
  test_text[i]["id"] = pred_test_corpus[i]
  line = test_text[i]['word']+'.'+test_text[i]['pos']+" | "+str(test_text[i]['id'])+" | "+test_text[i]['examples']
  text_file.write(line)
text_file.close()

In [ ]:
len(file_dataframe('test_corpus_lesk.data'))

3918